# **SeisMIIC** Tutorial
In the following, we will go through a simple example to compute a ambient noise correlations and monitor velocity changes using **SeisMIIC**.

The main code is hosted here: [SeisMIIC](https://github.com/PeterMakus/PyGLImER).

The documentation, which this notebook is based upon is located here:
[SeisMIIC Documentation](https://petermakus.github.io/PyGLImER/).

Without further ado, we'll dive right into it starting with data download.

## 1. Download the raw data
**SeisMIIC** uses [obspy](https://docs.obspy.org/) to download data from FDSN servers. Here, we will download data from station ``XD.LBAB`` for one year to invistigate seasonal changes in seismic velocity.

To download the data, we will use the `miic3.trace_data.waveform.Store_Client` class and its method `download_waveforms_mdl()`.

In [3]:
import os

from obspy.clients.fdsn import Client
from obspy import UTCDateTime

from miic3.trace_data.waveform import Store_Client

# Get notebook path for future reference of the database:
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

os.chdir(ipynb_path)

root = os.path.join(ipynb_path, 'data')
os.makedirs(root, exist_ok=True)

starttime = UTCDateTime(year=2001, julday=25)
endtime = UTCDateTime(year=2002, julday=1)
network = 'XD'
station = 'DELB'


c = Client('IRIS')
sc = Store_Client(c, root, read_only=False)
sc.download_waveforms_mdl(
    starttime, endtime, clients=[c], network=network,
    station=station, location='*', channel='BH?')


[2021-07-25 12:59:51,910] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for http://service.iris.edu.
[2021-07-25 12:59:51,911] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): http://service.iris.edu.
[2021-07-25 12:59:51,912] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2021-07-25 12:59:51,912] - obspy.clients.fdsn.mass_downloader - INFO: Client 'http://service.iris.edu' - Requesting reliable availability.
[2021-07-25 12:59:52,710] - obspy.clients.fdsn.mass_downloader - INFO: Client 'http://service.iris.edu' - Successfully requested availability (0.80 seconds)
[2021-07-25 12:59:52,764] - obspy.clients.fdsn.mass_downloader - INFO: Client 'http://service.iris.edu' - Found 1 stations (3 channels).
[2021-07-25 12:59:52,765] - obspy.clients.fdsn.mass_downloader - INFO: Client 'http://service.iris.edu' - Will attempt to download data from 1 stations.
[2021-07-25 12:59:52,793] - obspy.clie

Some notes about this:
1. The method `download_waveforms_mdl()` expects a list of clients as input.
2. All arguments accept wildcards

If everything worked fine. This should have created a folder called `data/mseed` and `data/inventory`. Let's check

In [7]:
os.listdir('./data/')

['inventory', 'mseed']

## 2. Compute Correlations
That seems to have worked, so we are ready to use this raw data to compute ambient noise correlations.

### 2.1 Parameters
Parameters are provided as a `yaml` file or a `dict` object. This tutorial comes with an yaml file to process the data. Let's have a short look at it using bash.

In [8]:
!cat params.yaml

#### Project wide parameters
# lowest level project directory
proj_dir : 'data/'
# directory for logging information
log_subdir : 'log'
# levels:
# 'DEBUG', 'INFO', 'WARNING', 'ERROR', or 'CRITICAL'
log_level: 'WARNING'
# folder for figures
fig_subdir : 'figures'
# folder containing preprocessed data


#### parameters that are network specific
net:
    # list of stations used in the project
    # type: list of strings or string, wildcards allowed
    network : 'XD'
    station : 'LBAB'
    # stations : ['D0.BZG', 'D0.ESO', 'D0.KBG', 'D0.KIR']
    # list of channels
    # type: list of strings
    channels : ['BHZ','BHN','BHE']

#### parameters for correlation (emperical Green's function creation)
co:
    # subdirectory of 'proj_dir' to store correlation
    # type: string
    subdir : 'corr'
    # times sequences to read for cliping or muting on stream basis
    # These should be long enough for the reference (e.g. the standard
    # deviation) to be rather independent of the parts to 

Each of the parameters is described in the [Documentation](url.to.doc.org).

To start the computation of the correlation we will use `MPI` and a simple python script, which could look like this:

In [9]:
!cat correlate.py

from time import time

from obspy.clients.fdsn import Client

from miic3.correlate.correlate import Correlator
from miic3.trace_data.waveform import Store_Client

# Path to the paramter file we created in the step before
params = 'params.yaml'
# You don't have to set this (could be None)
client = Client('IRIS')
# root is the same as proj_dir in params.yaml
root = 'data'
sc = Store_Client(client, root)

c = Correlator(sc, options=params)
print('Correlator initiated')
x = time()
st = c.pxcorr()
print('Correlation finished after', time()-x, 'seconds')

## 2.2 Start correlation
To start the correlation, we will use the `mpirun` command in bash:

In [18]:
import os
# This gives number of threads, usually twice as many as physical cores
ncpus = os.cpu_count()/2
!mpirun -n $ncpus python correlate.py

Correlator initiated
Correlator initiated
Correlator initiated
Correlator initiated
Correlator initiated
Correlator initiated
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loading locally ... Success
Loadin

Now let's have a look at those correlations. To do so, we use the `CorrelationDataBase` object.
All correlations are saved in the folder `data/corr` as defined in our params.yaml file.

In [19]:
from miic3.db.corr_hdf5 import CorrelationDataBase

with CorrelationDataBase('data/corr/XD-XD.DELB-DELB.h5', mode='r') as cdb:
    # find the available labels
    print(list(cdb.keys()))

['co', 'recombined', 'stack_86398', 'subdivision']


**SeisMIIC's** standard labels are `'subdivision` for the correlations of corr_len and `stack_*stacklen*` for the stack (with *stacklen* being the length of the stack in seconds).

**SeisMIIC** uses some sort of "combined seed codes" structured as above.

In [22]:
with CorrelationDataBase('data/corr/XD-XD.DELB-DELB.h5', mode='r') as cdb:
    # find the available labels
    print(cdb.get_available_channels(
        tag='stack_86398', network='XD-XD', station='DELB-DELB'))
    cst = cdb.get_data('XD-XD', 'DELB-DELB', 'BHE-BHN', 'stack_86398')

['BHE-BHN', 'BHE-BHZ', 'BHN-BHZ']
